# Chicago Crime Data SQL Analysis
This notebook loads datasets into SQLite and performs SQL queries for analysis.

In [1]:
import sqlite3
import pandas as pd

# Load datasets
census_df = pd.read_csv('ChicagoCensusData.csv')
crime_df = pd.read_csv('ChicagoCrimeData.csv')
schools_df = pd.read_csv('ChicagoPublicSchools.csv')

# Connect to SQLite
conn = sqlite3.connect('chicago.db')

# Load tables
census_df.to_sql('CENSUS_DATA', conn, if_exists='replace', index=False)
crime_df.to_sql('CRIME', conn, if_exists='replace', index=False)
schools_df.to_sql('CHICAGO_PUBLIC_SCHOOLS', conn, if_exists='replace', index=False)

print("Data loaded into SQLite database.")

Data loaded into SQLite database.


## Problem 1: Total number of crimes

In [2]:
query1 = "SELECT COUNT(*) AS TOTAL_CRIMES FROM CRIME"
result1 = pd.read_sql_query(query1, conn)
print(result1)

   TOTAL_CRIMES
0           533


## Problem 2: Community areas with per capita income less than 11000

In [3]:
query2 = "SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000"
result2 = pd.read_sql_query(query2, conn)
print(result2)

   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME
0                   26.0  West Garfield Park
1                   30.0      South Lawndale
2                   37.0         Fuller Park
3                   54.0           Riverdale


## Problem 3: Crimes involving minors (excluding 'children')

In [4]:
query3 = "SELECT CASE_NUMBER FROM CRIME WHERE DESCRIPTION LIKE '%MINOR%'"
result3 = pd.read_sql_query(query3, conn)
print(result3)

  CASE_NUMBER
0    HL266884
1    HK238408


## Problem 4: Kidnapping crimes involving a child

In [5]:
query4 = "SELECT * FROM CRIME WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%'"
result4 = pd.read_sql_query(query4, conn)
print(result4)

        ID CASE_NUMBER        DATE                 BLOCK  IUCR PRIMARY_TYPE  \
0  5276766    HN144152  2007-01-26  050XX W VAN BUREN ST  1792   KIDNAPPING   

                DESCRIPTION LOCATION_DESCRIPTION  ARREST  DOMESTIC  ...  \
0  CHILD ABDUCTION/STRANGER               STREET       0         0  ...   

   DISTRICT  WARD  COMMUNITY_AREA_NUMBER  FBICODE X_COORDINATE  Y_COORDINATE  \
0        15  29.0                   25.0       20    1143050.0     1897546.0   

   YEAR   LATITUDE  LONGITUDE                       LOCATION  
0  2007  41.874908 -87.750249  (41.874908413, -87.750249307)  

[1 rows x 21 columns]


## Problem 5: Types of crimes recorded at schools

In [6]:
query5 = "SELECT DISTINCT PRIMARY_TYPE FROM CRIME WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'"
result5 = pd.read_sql_query(query5, conn)
print(result5)

             PRIMARY_TYPE
0                 BATTERY
1         CRIMINAL DAMAGE
2               NARCOTICS
3                 ASSAULT
4       CRIMINAL TRESPASS
5  PUBLIC PEACE VIOLATION


## Problem 6: Type of schools with average safety score

In [7]:
query6 = "SELECT [Elementary, Middle, or High School], AVG(SAFETY_SCORE) AS AVERAGE_SAFETY_SCORE FROM CHICAGO_PUBLIC_SCHOOLS GROUP BY [Elementary, Middle, or High School]"
result6 = pd.read_sql_query(query6, conn)
print(result6)

  Elementary, Middle, or High School  AVERAGE_SAFETY_SCORE
0                                 ES             49.520384
1                                 HS             49.623529
2                                 MS             48.000000


## Problem 7: 5 community areas with highest % of households below poverty line

In [8]:
query7 = "SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5"
result7 = pd.read_sql_query(query7, conn)
print(result7)

   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME  PERCENT_HOUSEHOLDS_BELOW_POVERTY
0                   54.0           Riverdale                              56.5
1                   37.0         Fuller Park                              51.2
2                   68.0           Englewood                              46.6
3                   29.0      North Lawndale                              43.1
4                   27.0  East Garfield Park                              42.4


## Problem 8: Most crime prone community area number

In [9]:
query8 = "SELECT COMMUNITY_AREA_NUMBER FROM CRIME WHERE COMMUNITY_AREA_NUMBER IS NOT NULL GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1"
result8 = pd.read_sql_query(query8, conn)
print(result8)

   COMMUNITY_AREA_NUMBER
0                   25.0


## Problem 9: Name of community area with highest hardship index (Sub-query)

In [10]:
query9 = "SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA)"
result9 = pd.read_sql_query(query9, conn)
print(result9)

  COMMUNITY_AREA_NAME
0           Riverdale


## Problem 10: Community Area Name with most crimes (Sub-query)

In [11]:
query10 = "SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER FROM CRIME WHERE COMMUNITY_AREA_NUMBER IS NOT NULL GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1)"
result10 = pd.read_sql_query(query10, conn)
print(result10)

conn.close()

  COMMUNITY_AREA_NAME
0              Austin
